# Create a time-of-flight lookup table for BIFROST

In [ ]:
from ess.reduce import time_of_flight
import sciline
import scipp as sc
from scippneutron.chopper import DiskChopper
from ess.spectroscopy.indirect.workflow import load_everything
from ess.bifrost.data import simulated_elastic_incoherent_with_phonon

## Load data

In [ ]:
input_filename = simulated_elastic_incoherent_with_phonon()
sample, triplets, _, raw_choppers, monitors, _ = load_everything(input_filename)

In [ ]:
source_position = next(iter(triplets.values())).coords['source_position']
sample_position = sample['position']

## Process choppers

In [ ]:
def parse_choppers(raw_choppers, source_position):
    choppers = sc.DataGroup()
    for name, chopper in raw_choppers.items():
        processed = chopper.copy()
        # These are constant in the simulated data.
        processed['rotation_speed'] = processed['rotation_speed'].data.mean()
        processed['phase'] = processed['phase'].data.mean()
        # Guessing here as this is not stored in the file.
        processed['beam_position'] = sc.scalar(0.0, unit='deg')
        # ess.reduce.time_of_flight.simulate_beamline requires the
        # positions to be relative to the source.
        processed['position'] = chopper['position'] - source_position
        choppers[name] = DiskChopper.from_nexus(processed)
    return choppers

In [ ]:
choppers = parse_choppers(raw_choppers, source_position)

## Setting up the workflow

The table needs to cover the range of distances for both sample and monitor.
Here, the shifts of 0.1m are arbitrarily chosen to make sure the table covers `l1` and `l_monitor`.

In [ ]:
norm_monitor = monitors['110_frame_3']
l_monitor = sc.norm(source_position - norm_monitor.coords['position'])
l1 = sc.norm(source_position - sample_position)
l_min = l_monitor - sc.scalar(0.1, unit='m')
l_max = l1 + sc.scalar(0.1, unit='m')

In [ ]:
simulation_results = time_of_flight.simulate_beamline(
    choppers=choppers, neutrons=5_000_000
)

In [ ]:
wf = sciline.Pipeline(
    time_of_flight.providers(),
    params={
        **time_of_flight.default_parameters(),
        time_of_flight.LtotalRange: (l_min, l_max),
        time_of_flight.SimulationResults: simulation_results,
        time_of_flight.DistanceResolution: sc.scalar(0.1, unit='m'),
    },
)

## Compute the table

In [ ]:
table = wf.compute(time_of_flight.TimeOfFlightLookupTable)
table

In [ ]:
table.squeeze().plot()

## Save to file

In [ ]:
table.save_hdf5('BIFROST-simulation-tof-lookup-table.h5')